In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed May  1 22:27:36 2019

@author: Bhumika_Patoliya
"""
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import pandas as pd

np.random.seed(400)

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
stemmer = SnowballStemmer("english")


path = "C:\\Users\\patol\\Desktop\\MS CS\\sem 3\\BIA 660\\Project\\revised_midterm_df.csv"
orig_df = pd.read_csv(path)
orig_df['raw_text'] = orig_df['review_text'].apply(lambda x: [y.strip().lower() for y in x.lower().replace("dr.","doctor ").replace("\r",". ").replace("\n",'. ').split(". ") if len(y.strip()) > 1][1:])

orig_grp = orig_df.groupby(['biz_name','link'])['raw_text'].sum().reset_index()
temp = orig_grp[['raw_text']].sum()['raw_text']


In [3]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(text) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [4]:
document_num = 50
doc_sample = 'This disk has failed many times. I would like to get it replaced.'

print("Original document: ")
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print("\n\nTokenized and lemmatized document: ")
print(preprocess(doc_sample))



processed_docs = [preprocess(doc) for doc in temp]


Original document: 
['This', 'disk', 'has', 'failed', 'many', 'times.', 'I', 'would', 'like', 'to', 'get', 'it', 'replaced.']


Tokenized and lemmatized document: 
['disk', 'fail', 'time', 'like', 'replac']


In [5]:
dictionary = gensim.corpora.Dictionary(processed_docs)

dictionary.filter_extremes(no_below=15, no_above=0.2, keep_n= 100000)


In [6]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]


In [7]:
lda_model =  gensim.models.LdaMulticore(bow_corpus, 
                                   num_topics = 10, 
                                   id2word = dictionary,                                    
                                   passes = 50,
                                   workers = 4)


In [8]:
for idx, topic in lda_model.print_topics(-1):
    print("Topic: {} \nWords: {}".format(idx, topic ))
    print("\n")
    

Topic: 0 
Words: 0.132*"nurs" + 0.074*"help" + 0.056*"rude" + 0.050*"desk" + 0.046*"servic" + 0.042*"nice" + 0.041*"staff" + 0.037*"receptionist" + 0.029*"horribl" + 0.027*"girl"


Topic: 1 
Words: 0.061*"money" + 0.042*"differ" + 0.038*"away" + 0.035*"billing" + 0.034*"locat" + 0.034*"happi" + 0.032*"pay" + 0.026*"close" + 0.025*"night" + 0.022*"show"


Topic: 2 
Words: 0.123*"year" + 0.079*"kid" + 0.071*"pediatrician" + 0.066*"see" + 0.046*"children" + 0.041*"famili" + 0.032*"doctor" + 0.029*"go" + 0.026*"chang" + 0.019*"move"


Topic: 3 
Words: 0.147*"know" + 0.045*"send" + 0.043*"day" + 0.030*"hand" + 0.028*"wrong" + 0.026*"prescript" + 0.026*"later" + 0.023*"follow" + 0.021*"home" + 0.020*"say"


Topic: 4 
Words: 0.087*"want" + 0.046*"leav" + 0.046*"child" + 0.043*"hospit" + 0.031*"vaccin" + 0.020*"walk" + 0.016*"start" + 0.015*"school" + 0.015*"sign" + 0.015*"tell"


Topic: 5 
Words: 0.196*"time" + 0.142*"wait" + 0.084*"hour" + 0.057*"minut" + 0.043*"long" + 0.037*"appoint" + 0.0